# Extract phenotype data from ICD10 for PheWAS analysis

## As-Is Software Disclaimer

This notebook is delivered "As-Is". Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

[MIT License](https://github.com/dnanexus/UKB_RAP/blob/main/LICENSE) applies to this notebook.

## Jupyterlab app details (launch configuration)

Recommended configuration
- Runtime: ~20 min
- Cluster configuration: `Single Node`
- Recommended instance: `mem2_ssd1_v2_x32`
- Cost: ~£0.5

## Dependencies
|Library |License|
|:------------- |:-------------|
|[pandas](https://pandas.pydata.org/) |[BSD-3](https://github.com/pandas-dev/pandas/blob/main/LICENSE)|
|[numpy](https://numpy.org/) |[BSD-3](https://github.com/numpy/numpy/blob/main/LICENSE.txt)|
|[bgenix](https://enkre.net/cgi-bin/code/bgen/doc/trunk/doc/wiki/bgenix.md) | [Boost Software License (MIT-like)](https://enkre.net/cgi-bin/code/bgen/file?name=LICENSE_1_0.txt&ci=trunk)|
|[PLINK2](https://www.cog-genomics.org/plink/2.0/) |[GPL](https://github.com/chrchang/plink-ng/blob/master/2.0/COPYING)|


## Introduction
This notebook:
- Retrieves ICD10 codes for samples
- Create phenotype table in long format for subsequent PheWAS analysis
- Cleans covariates file
- Extracts genetic data, one file per variant
- Uploads data to UKB RAP

## Prepare your environment

Uncomment the install commands if you are comfortable with the library license and want to install and run the parts notebook that depend on the library.

In [ ]:
%%capture captured
%%bash
# Install PLINK2
#cd /opt/notebooks
#wget https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220814.zip
#unzip -o plink2_linux_avx2_20220814.zip

In [ ]:
!./plink2 --version

In [ ]:
%%capture captured
%%bash
# Install bgenix
#cd /opt/notebooks
#wget http://code.enkre.net/bgen/tarball/release/bgen.tgz
#tar xvfz bgen.tgz > /dev/null
#cd bgen.tgz/
#./waf configure 
#./waf 
#./build/test/unit/test_bgen
#./build/apps/bgenix -g example/example.16bits.bgen –list
#cd /opt/notebooks

In [ ]:
import dxpy
import numpy as np
import pandas as pd
import re
import shutil
import subprocess
import glob
import os
import ast

## Load dataset description and select entity containing phenotypic data

In [ ]:
# Automatically discover dispensed dataset ID and load the dataset
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)
dispensed_dataset_id = dispensed_dataset["id"]

In [ ]:
# Get project ID
project_id = dxpy.find_one_project()["id"]

In [ ]:
dataset = (":").join([project_id, dispensed_dataset_id])

Using the `-ddd` parameter will extract 3 dictionary files associated with the dataset.

The 3 dictionary files that are returned include:
1. `entity_dictionary` that contains the different tables resources that are available. The table we’re most interested in tends to be the participant table that contains the information about each participant.
2. `data_dictionary` that contains the different field names that we might want to include in our dataset.
3. `coding_dictionary` that contains a lookup for the values for some of the field names.


*Note: The following cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run*


In [ ]:
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

Specify fields ID to retrieve.

- `eid` - Participant's ID
- `41270` - [Diagnoses - ICD10](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=41270)

## Retrieve data

In this section we:
1. Use the `data_dictionary.csv` table to filter the list of field names and select ones of interest (i.e. 'eid' and 'p41270').
2. Next we concatenate the field names with entity name (i.e. 'participant')
3. Finally, we use this list of field names joined with entity names (i.e. 'participant.eid') to query for a dataset with only those fields.

In [ ]:
field_ids = ["41270"]

In [ ]:
path = os.getcwd()

In [ ]:
data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

In [ ]:
# The UKB participant tables have the following naming convention for the fields (or columns): "p<field id>_i<instance id>_a<array id>"
# This function is used to grab all field names (e.g. "p<field_id>_iYYY_aZZZ") given a list of field IDs and return string to pass into
# extract_dataset
def fields_for_id(field_id):
    field_names = ["eid"]
    for _id in field_id:
        select_field_names = list(
            data_dict_df[
                data_dict_df.name.str.match(r"^p{}(_i\d+)?(_a\d+)?$".format(_id))
            ].name.values
        )
        field_names += select_field_names

    field_names = [f"participant.{f}" for f in field_names]
    return ",".join(field_names)

In [ ]:
field_names = fields_for_id(field_ids)
field_names

In [ ]:
# Load dataset
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
# Note: There is no space separating the different fields
cmd = [
    "dx",
    "extract_dataset",
    dataset,
    "--fields",
    field_names,
    "--delimiter",
    ",",
    "--output",
    "pheno_dictionary.csv",
]
subprocess.check_call(cmd)

In [ ]:
pheno_dict_csv = "pheno_dictionary.csv"
pheno = pd.read_csv(pheno_dict_csv)
pheno.head()

In [ ]:
# Rename column headers
pheno = pheno.rename(columns=lambda x: re.sub("participant.", "", x))
pheno.head()

In [ ]:
pheno.shape

## Create phenotype table in PheWAS format

Each row is containing one phenotype (ICD 10 diagnosis) for one participant. Therefore, when participant has multiple diagnoses, participants eid will appear multiple times.

In [ ]:
# Convert the values in the p41270 column from a string representation of a list to an actual list
# using ast.literal_eval()
pheno = pheno.set_index("eid").dropna()
pheno["p41270"] = pheno["p41270"].apply(lambda x: ast.literal_eval(x))
pheno.head()

In [ ]:
pheno_long_all = pheno.explode("p41270").reset_index()
pheno_long_all["count"] = 1
pheno_long_all["vocabulary_id"] = "ICD10"
pheno_long_all.rename({"eid": "id", "p41270": "code"}, axis=1, inplace=True)
pheno_long_all = pheno_long_all[["id", "vocabulary_id", "code", "count"]]
pheno_long_all.head()

In [ ]:
pheno_long_all.shape

### Save phenotype table as CSV file and upload it

In [ ]:
pheno_long_all.to_csv("pheno_icd10_long.csv", index=False)

In [ ]:
! dx upload pheno_icd10_long.csv --path /Data/PheWAS/

## Get covariate files

In [ ]:
!dx download -f /Data/ischemia_df.phe

In [ ]:
pheno = pd.read_csv("ischemia_df.phe", sep='\t')
pheno.head()

In [ ]:
pheno[['FID']].to_csv('eids_to_keep.txt', index=False, sep='\t', header=False)

In [ ]:
!dx upload eids_to_keep.txt --path /Data/

In [ ]:
covars = pheno.loc[:, [col for col in pheno.columns if col not in ['FID', 'ischemia_cc', 'hdl_cholesterol', 'ldl_cholesterol', 'hypertension']]]
covars.rename(columns={'IID': 'id'}, inplace=True)

In [ ]:
covars.to_csv('covariates.txt', index=False, sep='\t')

In [ ]:
!dx upload covariates.txt --path /Data/PheWAS/

## Get genetic data

Download results of LD clumping

In [ ]:
!dx download -f /Data/LD_clump/plink_all_ld_clumped_ld_clumped.clumped

In [ ]:
ld_results = pd.read_csv('plink_all_ld_clumped_ld_clumped.clumped', delim_whitespace=True)
ld_results.sort_values(by='P').head()

In [ ]:
ld_results.shape

In [ ]:
%%bash
# Create symlink for imputed data
DIR='/mnt/project/Bulk/Imputation/Imputation*from*genotype*(GEL)'
ln -sf $DIR /opt/notebooks/imputed
DIR2=/mnt/project/Bulk-DRL/GEL_imputed_sample_files_fixed/
ln -sf $DIR2 /opt/notebooks/samples

For each varinat in LD table, extract genotypes and create PLINK raw file.

In [ ]:
if os.path.exists('phewas_geno_data'):
    shutil.rmtree('phewas_geno_data')

os.mkdir('phewas_geno_data')

for rsid in ld_results['SNP'].values:
    chromosome = ld_results[ld_results['SNP'] == rsid]['CHR'].values[0]
    #rsid = 'rs10455872'
    print(f'rsID: {rsid}, chromosome: {chromosome}')
    new_bgen_name = f'phewas_geno_data/{rsid}.bgen'
    plink_output_prefix = f'phewas_geno_data/plink_{rsid}'
    with open(new_bgen_name, 'wb') as new_bgen:
        print(f'Extract significant rsIDs')
        subprocess.check_call(['/opt/notebooks/bgen.tgz/build/apps/bgenix', '-g', f'imputed/ukb21008_c{chromosome}_b0_v1.bgen', 
                        '-incl-rsids', rsid], stdout=new_bgen, stderr=subprocess.PIPE)
        print(f'Make PLINK files')
        subprocess.check_call(['./plink2', '--bgen', new_bgen_name, 'ref-first', '--sample', 
                           f'samples/ukb21008_c{chromosome}_b0_v1.sample', '--rm-dup', 'force-first', '--out', plink_output_prefix,
                           '--keep-fam', 'eids_to_keep.txt', '--export',  'A'])

Open each raw PLINK file and rename header to be in PheWAS format.

In [ ]:
if os.path.exists('phewas_geno_rsid'):
    shutil.rmtree('phewas_geno_rsid')

os.mkdir('phewas_geno_rsid')

for file in glob.glob('phewas_geno_data/*.raw'):
    print(file.split('/')[1])
    dosage = pd.read_csv(file, sep='\t')
    dosage = dosage.iloc[:, [0,-1]]
    dosage.rename(columns={dosage.columns[0]: 'id', dosage.columns[1]: dosage.columns[1].split('_')[0]}, inplace = True)
    dosage.to_csv(f'phewas_geno_rsid/{dosage.columns[1]}.txt', index=False, sep='\t')

Upload data to UKB RAP

In [ ]:
!dx upload -r phewas_geno_rsid/ --path /Data/PheWAS/phewas_geno_rsid/

## Output files

- Phenome data in long format containing ICD 10 codes per participant (`pheno_icd10_long.csv`)
- Covariates file (`covariates.txt`)
- List of participants EIDs to include in the analysis (`eids_to_keep.txt`)
- Table for each index variant in LD clumping containing 2 columns: participant ID and rsID genotyping results (`<rsid>.raw`)